In [1]:
import numpy as np

In [72]:
# Game Specifications
board = [[" " for i in range(10)] for i in range(10)]

top = [None, None, 112, None, 48, 3087, 9, None, None, None]
right = [None, 4, 27, None, None, None, 16, None, None, None]
bottom = [2025, None, None, 12, 64, 5, None, 405, None, None]
left = [None, None, None, 27, None, None, None, 12, 225, None]

In [ ]:
# get all possible factor decompositions. 
# 1 can only be at the beginning or end because no mirrors are allowed to be in orthorgonal adjacent fields

In [67]:
def get_all_factors_rec(x):
    
    if x == 1:
        return [[1]]
    
    combs = []
    for i in range(2, x+1):
        
        if x == i:
            combs.append([x])
        
        elif x % i == 0:
            sub_combs = get_all_factors_rec(int(x/i))
            
            for comb in sub_combs:
                comb.append(i)
                combs.append(comb)
                
    return combs

def get_all_factors(x):
    
    combs = get_all_factors_rec(x)
    final_combs = []
    for comb in combs:
        final_combs.append(comb)
        final_combs.append(comb + [1])
        final_combs.append([1] + comb)
        final_combs.append([1] + comb + [1])

    return final_combs

In [98]:
left_turn = {
    "d" : "l",
    "u" : "r",
    "r" : "d",
    "l" : "u"
}

right_turn = {
    "d" : "r",
    "u" : "l",
    "r" : "u",
    "l" : "d"
}  

In [187]:
mirror_type = {
    "u" : {
        "l" : "\\",
        "r" : "//"
    },
    "d" : {
        "l" : "//",
        "r" : "\\"
    },
    "l" : {
        "u" : "\\",
        "d" : "//"
    },
    "r" : {
        "u" : "//",
        "d" : "\\"
    },
}

In [139]:
def in_field(pos):
    if not (pos[0] >= 0 and pos[0] < 10):
        return False
    
    if not (pos[1] >= 0 and pos[1] < 10):
        return False

    return True

In [140]:
def step(pos, ori, num):
    if ori == "d":
        return [pos[0] + num, pos[1]]
    if ori == "u":
        return [pos[0] - num, pos[1]]
    if ori == "r":
        return [pos[0], pos[1] + num]
    if ori == "l":
        return [pos[0], pos[1] - num]

In [156]:
def get_num(pos):
    if pos[0] == -1:
        return top[pos[1]]
    elif pos[0] == 10:
        return bottom[pos[1]]
    elif pos[1] == -1:
        return left[pos[0]]
    elif pos[1] == 10:
        return right[pos[0]]

In [ ]:
# Get all possible paths of a laser that starts at pos, where the segements must have the lengths in combs
# and the product of these segment lengths has to be num

In [241]:
def get_possible_paths_rec(pos, ori, comb, idx):
    
    paths = []
    if len(comb) == idx:
        if pos[0] == -1 or pos[1] == -1 or pos[0] == 10 or pos[1] == 10:
            return [[(pos, None)]]
        else:
            return []
    
    elif not in_field(pos):
        return []   
    
    else:
        
        ori_left = left_turn[ori]
        pos_left = step(pos, ori_left, comb[idx])
        paths_left = get_possible_paths_rec(pos_left, ori_left, comb, idx+1)
        
        for path in paths_left:
            paths.append([(pos, mirror_type[ori][ori_left])] + path)
        
        
        ori_right = right_turn[ori]
        pos_right = step(pos, ori_right, comb[idx])
        paths_right = get_possible_paths_rec(pos_right, ori_right, comb, idx+1)
        for path in paths_right:
            paths.append([(pos, mirror_type[ori][ori_right])] + path)
        
        
        
        return paths
        
def get_possible_paths(pos, num, comb):
    
    if pos[0] == -1:
        ori = "d"
    elif pos[0] == 10:
        ori = "u"
    elif pos[1] == -1:
        ori = "r"
    elif pos[1] == 10:
        ori = "l"
    
    paths = []
    next_pos = step(pos, ori, comb[0])
    next_paths = get_possible_paths_rec(next_pos, ori, comb, 1)
    
    for path in next_paths:
        last_pos = path[-1][0]
        last_num = get_num(last_pos)
        
        if last_num == num or last_num is None:
            paths.append([(pos, None)] + path)
    
    return paths

def get_all_possible_paths(pos, num):
    combs = get_all_factors(num)
    
    all_paths = []
    for comb in combs:
        paths = get_possible_paths(pos, num, comb)
        if paths != []:
            all_paths += paths
    return all_paths
        

In [236]:
def exc_range(a,b):
    rang = []
    if a < b:
        diff = 1
    else:
        diff = -1
    
    a += diff
    while a != b:
        rang.append(a)
        a += diff
        
    return rang
    

In [493]:
# This function adds a path to a game_state
# Mirrors are displayed as // or \\
# Every field the laser passes through is blocked with "b " so that later paths can't put mirrors in these
# fields

In [375]:
def add_path(game_state, path):
    
        # block between
        for i in range(1,len(path)):
            if path[i-1][0][0] != path[i][0][0]:
                # change in the first coordinate
                y = path[i][0][1]
                for j in exc_range(path[i-1][0][0], path[i][0][0]):
                    
                    if game_state[j][y] == "//" or game_state[j][y] == "\\":
                        return False
                    game_state[j][y] = "b "
            else:
                x = path[i][0][0]
                for j in exc_range(path[i-1][0][1], path[i][0][1]):
                    
                    if game_state[x][j] == "//" or game_state[x][j] == "\\":
                        return False
                    game_state[x][j] = "b "
        
        # set mirrors
        for i in range(1,len(path)-1):
            pos = path[i][0]
            mirror = path[i][1]
            field = game_state[pos[0]][pos[1]]
            if field == 'b ' or (field != '  ' and field != mirror):
                return False
            
            game_state[pos[0]][pos[1]] = mirror
        
        
        return True

In [494]:
# For each laser with a number determine all possible paths

In [463]:
laser_to_paths = {}
laser_id = 0
for i in range(len(top)):
    if not (top[i] is None):
        all_paths = get_all_possible_paths([-1,i], top[i])
        laser_to_paths[laser_id] = []
        for path in all_paths:
            game_state = [["  " for i in range(10)] for i in range(10)]
            if add_path(game_state, path):
                laser_to_paths[laser_id].append(path)
        laser_id += 1

for i in range(len(right)):
    if not (right[i] is None):
        all_paths = get_all_possible_paths([i,10], right[i])
        laser_to_paths[laser_id] = []
        for path in all_paths:
            game_state = [["  " for i in range(10)] for i in range(10)]
            if add_path(game_state, path):
                laser_to_paths[laser_id].append(path)
        laser_id += 1

for i in range(len(bottom)):
    if not (bottom[i] is None):
        all_paths = get_all_possible_paths([10,i], bottom[i])
        laser_to_paths[laser_id] = []
        for path in all_paths:
            game_state = [["  " for i in range(10)] for i in range(10)]
            if add_path(game_state, path):
                laser_to_paths[laser_id].append(path)
        laser_id += 1

for i in range(len(left)):
    if not (left[i] is None):
        all_paths = get_all_possible_paths([i, -1], left[i])
        laser_to_paths[laser_id] = []
        for path in all_paths:
            game_state = [["  " for i in range(10)] for i in range(10)]
            if add_path(game_state, path):
                laser_to_paths[laser_id].append(path)
        laser_id += 1

In [438]:
import copy

In [495]:
# brute force all possible path combinations

In [468]:
def check_possible(state, laser_id):
    if laser_id == 15:
        print(f"{state}")
        return True
    
    for path in laser_to_paths[laser_id]:
        tmp_state = copy.deepcopy(state)
        succ = add_path(tmp_state, path)
        
            
        if succ and check_possible(tmp_state, laser_id+1):
            return True
        
    return False

In [469]:
game_state = [["  " for i in range(10)] for i in range(10)]

In [470]:
check_possible(game_state, 0)

[['b ', 'b ', '\\', 'b ', 'b ', 'b ', '\\', 'b ', 'b ', '\\'], ['b ', 'b ', 'b ', 'b ', 'b ', 'b ', 'b ', 'b ', '\\', 'b '], ['//', 'b ', 'b ', 'b ', '//', 'b ', 'b ', '//', 'b ', 'b '], ['b ', 'b ', '//', 'b ', 'b ', 'b ', 'b ', 'b ', 'b ', '\\'], ['b ', 'b ', 'b ', 'b ', 'b ', 'b ', '//', 'b ', 'b ', 'b '], ['b ', '//', 'b ', 'b ', '\\', 'b ', 'b ', '\\', 'b ', 'b '], ['//', 'b ', '\\', 'b ', 'b ', '//', 'b ', 'b ', '//', 'b '], ['b ', 'b ', 'b ', '\\', 'b ', 'b ', 'b ', 'b ', 'b ', 'b '], ['b ', 'b ', 'b ', 'b ', '//', 'b ', '//', 'b ', '//', 'b '], ['//', 'b ', 'b ', 'b ', 'b ', '//', 'b ', 'b ', 'b ', 'b ']]


True

In [472]:
solution = [['b ', 'b ', '\\', 'b ', 'b ', 'b ', '\\', 'b ', 'b ', '\\'], ['b ', 'b ', 'b ', 'b ', 'b ', 'b ', 'b ', 'b ', '\\', 'b '], ['//', 'b ', 'b ', 'b ', '//', 'b ', 'b ', '//', 'b ', 'b '], ['b ', 'b ', '//', 'b ', 'b ', 'b ', 'b ', 'b ', 'b ', '\\'], ['b ', 'b ', 'b ', 'b ', 'b ', 'b ', '//', 'b ', 'b ', 'b '], ['b ', '//', 'b ', 'b ', '\\', 'b ', 'b ', '\\', 'b ', 'b '], ['//', 'b ', '\\', 'b ', 'b ', '//', 'b ', 'b ', '//', 'b '], ['b ', 'b ', 'b ', '\\', 'b ', 'b ', 'b ', 'b ', 'b ', 'b '], ['b ', 'b ', 'b ', 'b ', '//', 'b ', '//', 'b ', '//', 'b '], ['//', 'b ', 'b ', 'b ', 'b ', '//', 'b ', 'b ', 'b ', 'b ']]
solution

[['b ', 'b ', '\\', 'b ', 'b ', 'b ', '\\', 'b ', 'b ', '\\'],
 ['b ', 'b ', 'b ', 'b ', 'b ', 'b ', 'b ', 'b ', '\\', 'b '],
 ['//', 'b ', 'b ', 'b ', '//', 'b ', 'b ', '//', 'b ', 'b '],
 ['b ', 'b ', '//', 'b ', 'b ', 'b ', 'b ', 'b ', 'b ', '\\'],
 ['b ', 'b ', 'b ', 'b ', 'b ', 'b ', '//', 'b ', 'b ', 'b '],
 ['b ', '//', 'b ', 'b ', '\\', 'b ', 'b ', '\\', 'b ', 'b '],
 ['//', 'b ', '\\', 'b ', 'b ', '//', 'b ', 'b ', '//', 'b '],
 ['b ', 'b ', 'b ', '\\', 'b ', 'b ', 'b ', 'b ', 'b ', 'b '],
 ['b ', 'b ', 'b ', 'b ', '//', 'b ', '//', 'b ', '//', 'b '],
 ['//', 'b ', 'b ', 'b ', 'b ', '//', 'b ', 'b ', 'b ', 'b ']]

In [419]:
game_state = [["  " for i in range(10)] for i in range(10)]

In [ ]:
# Determine the missing number on the edge of the board and calculate the final solution

In [485]:
def follow_laser(game_state, pos, x_diff, y_diff):
    
    seg_len = 1
    pos[0] += x_diff
    pos[1] += y_diff
    
    while in_field(pos) and game_state[pos[0]][pos[1]] == "b ":
        pos[0] += x_diff
        pos[1] += y_diff
        seg_len+=1
    
    if not in_field(pos):
        return seg_len
    
    # change direction
    if game_state[pos[0]][pos[1]] == "\\":
        tmp = x_diff
        x_diff = y_diff
        y_diff = tmp
        
    elif game_state[pos[0]][pos[1]] == "//":
        tmp = x_diff
        x_diff = -y_diff
        y_diff = -tmp
        
    return seg_len * follow_laser(game_state, pos, x_diff, y_diff)
    

In [486]:
follow_laser(solution, [-1,2], 1, 0)

112

In [491]:
def fill_edges(state):
    
    # top
    top_edge = []
    for i in range(10):
        top_edge.append(follow_laser(state, [-1,i], 1, 0))
        
    print(f"top edge: {top_edge}")
    
    right_edge = []
    for i in range(10):
        right_edge.append(follow_laser(state, [i, 10], 0, -1))
        
    print(f"rig edge: {right_edge}")
    
    bottom_edge = []
    for i in range(10):
        bottom_edge.append(follow_laser(state, [10, i], -1, 0))
        
    print(f"bot edge: {bottom_edge}")
    
    left_edge = []
    for i in range(10):
        left_edge.append(follow_laser(state, [i, -1], 0, 1))
        
    print(f"lef edge: {left_edge}")
    
    
    final_solution = sum(top_edge) * sum(right_edge) * sum(bottom_edge) * sum(left_edge)
    
    print(f"final_solution: {final_solution}")
    

In [492]:
fill_edges(solution)

top edge: [3, 12, 112, 56, 48, 3087, 9, 405, 4, 1]
rig edge: [1, 4, 27, 9, 64, 27, 16, 56, 4, 5]
bot edge: [2025, 225, 24, 12, 64, 5, 16, 405, 4, 3087]
lef edge: [27, 2025, 3, 27, 112, 12, 48, 12, 225, 24]
final_solution: 11745101625405
